In [ ]:
# default_exp plugin.schema

In [ ]:
# export
# hide
import random, string
from typing import Optional

from pymemri.data.itembase import Item

In [ ]:
# export
# hide
class Account(Item):

    properties = Item.properties + [
        "service",
        "identifier",
        "secret",
        "code",
        "refreshToken",
        "errorMessage",
    ]
    edges = Item.edges

    def __init__(
        self,
        service: str = None,
        identifier: str = None,
        secret: str = None,
        code: str = None,
        refreshToken: str = None,
        errorMessage: str = None,
        **kwargs
    ):
        super().__init__(**kwargs)
        
        # Properties
        self.service: Optional[str] = service
        self.identifier: Optional[str] = identifier
        self.secret: Optional[str] = secret
        self.refreshToken: Optional[str] = refreshToken
        self.code: Optional[str] = code
        self.errorMessage: Optional[str] = errorMessage

In [ ]:
# export
# hide
class PluginRun(Item):
    properties = Item.properties + [
        "containerImage",
        "pluginModule",
        "pluginName",
        "status",
        "targetItemId",
        "authUrl",
        "error",
        "settings",
    ]
    edges = Item.edges + ["view", "persistentState", "account"]

    def __init__(
        self,
        containerImage: str = None,
        pluginModule: str = None,
        pluginName: str = None,
        status: str = None,
        settings: str = None,
        targetItemId: str = None,
        authUrl: str = None,
        error: str = None,
        persistentState: list = None,
        account: list = None,
        view: list = None,
        **kwargs
    ):
        super().__init__(**kwargs)
        id_ = (
            "".join([random.choice(string.hexdigits) for i in range(32)])
            if targetItemId is None
            else targetItemId
        )
        
        # Properties
        self.pluginModule: Optional[str] = pluginModule
        self.pluginName: Optional[str] = pluginName
        self.containerImage: Optional[str] = containerImage
        self.targetItemId: Optional[str] = id_
        self.id: Optional[str] = id_
        self.status: Optional[str] = status 
        self.settings: Optional[str] = settings
        self.authUrl: Optional[str] = authUrl
        self.error: Optional[str] = error
        
        # Edges
        self.account: list = account if account is not None else []
        self.persistentState: list = persistentState if persistentState is not None else []
        self.view: list = view if view is not None else []

In [ ]:
run = PluginRun("image", "module", "name", account=[Account()])

In [ ]:
# export
# hide
class PersistentState(Item):
    # TODO remove this schema, it is no longer needed with the updated PluginRun

    properties = Item.properties + ["pluginId", "state"]
    edges = Item.edges + ["account", "view"]

    def __init__(self, pluginName=None, state=None, account=None, view=None, **kwargs):
        super().__init__(**kwargs)
        self.pluginName = pluginName
        self.status = state
        self.account = account if account is not None else []
        self.view = view if view is not None else []

    def get_state(self):
        return self.status

    def set_state(self, client, state_str):
        self.status = state_str
        client.update_item(self)

    def get_account(self):
        if len(self.account) == 0:
            return None
        else:
            return self.account[0]

    def set_account(self, client, account):
        if len(self.account) == 0:
            if not account.id:
                client.create(account)
            self.add_edge("account", account)
            self.update(client)
        else:
            existing_account = self.account[0]
            for prop in account.properties:
                value = getattr(account, prop, None)
                if value and hasattr(existing_account, prop):
                    setattr(existing_account, prop, value)
            existing_account.update(client)

    def get_view_by_name(self, view_name):
        for cvu in self.view:
            if cvu.name == view_name:
                return cvu

    def set_views(self, client, views=None):
        for view in views:
            client.create(view)
            self.add_edge("view", view)
        self.update(client)
        return True